In [25]:
import pandas as pd
import numpy as np
import os
import re
from icecream import ic
from collections import defaultdict

In [26]:
import warnings
warnings.filterwarnings("ignore")

In [27]:
import toml
x = toml.load('/home/skowshik/ADRD_repo/pipeline_v1_main/adrd_tool/dev/data/toml_files/default_conf_new.toml')
y = toml.load('/home/skowshik/ADRD_repo/pipeline_v1_main/adrd_tool/dev/data/toml_files/tmp.toml')

In [28]:
len(x['feature'])

391

In [29]:
for k,v in x['feature'].items():
    if v != y['feature'][k]:
        print(k, v)
        print(y['feature'][k])

his_BIRTHMO {'type': 'categorical', 'num_categories': 13}
{'type': 'categorical', 'num_categories': 12}
bat_LOGIMO {'type': 'categorical', 'num_categories': 13}
{'type': 'categorical', 'num_categories': 12}
bat_LOGIDAY {'type': 'categorical', 'num_categories': 32}
{'type': 'categorical', 'num_categories': 28}


In [7]:
basedir = '../../data'
fname = 'training_cohorts/merged_data_nacc_nifd_stanford_aibl_ppmi_oasis_rtni.csv'
# fname = 'train_vld_test_split_updated/adni_revised_labels.csv'
data = pd.read_csv(f'{basedir}/{fname}')
# data = pd.read_csv('/home/skowshik/ADRD_repo/pipeline_v1_main/adrd_tool/data/train_vld_test_split_updated/bmc_revised.csv')
# data = pd.read_csv('/home/skowshik/ADRD_repo/pipeline_v1_main/adrd_tool/data/training_cohorts/new_nacc_revised_selection.csv')
meta1 = pd.read_csv("/home/skowshik/ADRD_repo/pipeline_v1_main/adrd_tool/dev/data/meta_files/tmp.csv")
# data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [8]:
data['his_MARISTAT'].value_counts()

0.0    28012
1.0     8473
2.0     5099
4.0     2288
5.0      743
3.0      471
Name: his_MARISTAT, dtype: int64

In [9]:
set(data.columns) - set(meta1['name'])

{'ID'}

In [10]:
len(data.columns)

464

In [11]:
meta1[meta1['name'] == 'his_EDUC']['type'].iloc[0]

'N'

In [12]:
'NC' in dict(meta1['name']).values()

True

In [13]:
meta1

,name,missing_rate,type,num_unique_values,length,unique_values
0,non_null_count,0.04,NaN,226,1,"[198.0, 146.0, 202.0, 259.0, 191.0, 200.0, 71...."
1,MRI_CNT,0.87,NaN,7,1,"[3.0, 4.0, 1.0, 5.0, 0.0, 2.0, 6.0]"
2,NC,0.00,C,2,1,"[0.0, 1.0]"
3,MCI,0.00,C,2,1,"[1.0, 0.0]"
4,DE,0.00,C,2,1,"[0.0, 1.0]"
...,...,...,...,...,...,...
458,bat_RESPOTHX,1.00,C,29,1,"['severe aphasia', 'headache', 'language barri..."
459,apoe_NACCNE4S,0.27,C,3,1,"[0.0, 2.0, 1.0]"
460,cdr_CDRGLOB,0.01,N,5,1,"[0.5, 0.0, 1.0, 3.0, 2.0]"
461,cdr_CDRSUM,0.02,N,33,1,"[0.5, 2.5, 1.5, 0.0, 5.5, 15.0, 6.5, 12.0, 2.0..."


In [14]:
meta = defaultdict()
percent_missing = dict(data.isnull().sum() / len(data))
for i, col in enumerate(data.columns):
    if col == 'filename' or col == 'ID':
        continue
    
    # print(col)
    # print(meta1[meta1['name'] == col]['type'].iloc[0])
    meta[col] = {}
    meta[col]['name'] = col
    meta[col]['unique_values'] = list(data[col].dropna().unique())
    meta[col]['num_unique_values'] = len(meta[col]['unique_values'])
    meta[col]['length'] = 1
    # if meta[col]['num_unique_values'] <= 4 or type(meta[col]['unique_values'][0]) == str:
    #     meta[col]['type'] = 'C'
    # else:
    #     meta[col]['type'] = 'N'
    if col in dict(meta1['name']).values():
        meta[col]['type'] = meta1[meta1['name'] == col]['type'].iloc[0]
    else:
        if meta[col]['num_unique_values'] == 2:
            meta[col]['type'] = 'C'
        else:
            meta[col]['type'] = 'N'
    meta[col]['missing_rate'] = round(percent_missing[col], 2)
# meta['img_MRI_T1'] = {'name': 'img_MRI_T1', 'type': 'M'}

In [15]:
df = pd.DataFrame(meta).T[['name', 'missing_rate', 'type', 'num_unique_values', 'length', 'unique_values']]

In [16]:
set(df['name']) - set(meta1['name'])

set()

In [17]:
df['type'].value_counts()

C    352
N    109
Name: type, dtype: int64

In [18]:
meta1['type'].value_counts()

C    352
N    109
Name: type, dtype: int64

In [19]:
df

,name,missing_rate,type,num_unique_values,length,unique_values
non_null_count,non_null_count,0.04,NaN,226,1,"[198.0, 146.0, 202.0, 259.0, 191.0, 200.0, 71...."
MRI_CNT,MRI_CNT,0.87,NaN,7,1,"[3.0, 4.0, 1.0, 5.0, 0.0, 2.0, 6.0]"
NC,NC,0.0,C,2,1,"[0.0, 1.0]"
MCI,MCI,0.0,C,2,1,"[1.0, 0.0]"
DE,DE,0.0,C,2,1,"[0.0, 1.0]"
...,...,...,...,...,...,...
bat_RESPOTHX,bat_RESPOTHX,1.0,C,29,1,"[severe aphasia, headache, language barrier, p..."
apoe_NACCNE4S,apoe_NACCNE4S,0.27,C,3,1,"[0.0, 2.0, 1.0]"
cdr_CDRGLOB,cdr_CDRGLOB,0.01,N,5,1,"[0.5, 0.0, 1.0, 3.0, 2.0]"
cdr_CDRSUM,cdr_CDRSUM,0.02,N,33,1,"[0.5, 2.5, 1.5, 0.0, 5.5, 15.0, 6.5, 12.0, 2.0..."


In [20]:
list(meta1[meta1['name'] == 'NC']['unique_values'])[0]

'[0.0, 1.0]'

In [21]:
import ast
for i, row in df.iterrows():
    # print(type(str(row['unique_values'])))
    # break
    if row['name'] in list(meta1['name']):
        a = sorted(set(row['unique_values']))
        b = sorted(set(ast.literal_eval(list(meta1[meta1['name'] == row['name']]['unique_values'])[0])))
        if -4 in a or -4 in b or '-4' in a or '-4' in b:
            print("found")
        # if row['type'] == 'C' and row['num_unique_values'] != list(meta1[meta1['name'] == row['name']]['num_unique_values'])[0] and not isinstance(row['unique_values'][0], str):
        #     # print("hello")
        #     print(row['name'])
        #     print(row['num_unique_values'])
        #     print(a)
        #     print(b)
        if a != b:
            if (a[0] >= b[0] and a[-1] <= b[-1]) or (a[0] >= 0 and a[-1] <= b[-1]):
                continue
            if isinstance(a[0], str):
                continue
            print(row['name'])
            print(a)
            print(b)
    # break
    


In [22]:
df.index

Index(['non_null_count', 'MRI_CNT', 'NC', 'MCI', 'DE', 'AD', 'LBD', 'PDD',
       'DLB', 'VD',
       ...
       'bat_RESPDISN', 'bat_RESPFATG', 'bat_RESPEMOT', 'bat_RESPASST',
       'bat_RESPOTH', 'bat_RESPOTHX', 'apoe_NACCNE4S', 'cdr_CDRGLOB',
       'cdr_CDRSUM', 'mri_zip'],
      dtype='object', length=463)

In [23]:
'exam_RESTTRL' in df.index

True

In [24]:
df.to_csv('meta_files/meta_file.csv', index=False)